In [1]:
import findspark
findspark.init()
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf
from pyspark import SQLContext
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.classification import LogisticRegression,MultilayerPerceptronClassifier
from pyspark.sql.types import IntegerType
sc = SparkContext("local", "first")
logFile = "file:///F:/FBDP/实验/实验四/logfile1.txt"  
logData = sc.textFile(logFile).cache()
spark = SparkSession.builder.config("spark.driver.memory", "16g").getOrCreate()

In [2]:
from pyspark.sql.functions import *
df=spark.read.options(header='True') .csv("file:///F:/FBDP/实验/实验四/train_data.csv")
df = df.na.fill(-1)
df = df.na.fill('-1')
print(type(df))
df=df.withColumnRenamed("is_default", "label")

<class 'pyspark.sql.dataframe.DataFrame'>


In [3]:
#class
class_indexer = StringIndexer(inputCol='class', outputCol='class_num').fit(df)
df = class_indexer.transform(df)
class_onehoter = OneHotEncoder(inputCol='class_num', outputCol='class_vector')
df = class_onehoter.transform(df)
#sub_class
sub_class_indexer = StringIndexer(inputCol='sub_class', outputCol='sub_class_num').fit(df)
df = sub_class_indexer.transform(df)
sub_class_onehoter = OneHotEncoder(inputCol='sub_class_num', outputCol='sub_class_vector')
df = sub_class_onehoter.transform(df)
#work_type
work_type_indexer = StringIndexer(inputCol='work_type', outputCol='work_type_num').fit(df)
df = work_type_indexer.transform(df)
work_type_onehoter = OneHotEncoder(inputCol='work_type_num', outputCol='work_type_vector')
df = work_type_onehoter.transform(df)
#employer_type
employer_type_indexer = StringIndexer(inputCol='employer_type', outputCol='employer_type_num').fit(df)
df = employer_type_indexer.transform(df)
employer_type_onehoter = OneHotEncoder(inputCol='employer_type_num', outputCol='employer_type_vector')
df = employer_type_onehoter.transform(df)
#industry
industry_indexer = StringIndexer(inputCol='industry', outputCol='industry_num').fit(df)
df = industry_indexer.transform(df)
industry_onehoter = OneHotEncoder(inputCol='industry_num', outputCol='industry_vector')
df = industry_onehoter.transform(df)
#work_year
work_year_indexer = StringIndexer(inputCol='work_year', outputCol='work_year_num').fit(df)
df = work_year_indexer.transform(df)
work_year_onehoter = OneHotEncoder(inputCol='work_year_num', outputCol='work_year_vector')
df = work_year_onehoter.transform(df)
#df.show(3)
#issue_date
issue_date_indexer = StringIndexer(inputCol='issue_date', outputCol='issue_date_num').fit(df)
df = issue_date_indexer.transform(df)
issue_date_onehoter = OneHotEncoder(inputCol='issue_date_num', outputCol='issue_date_vector')
df = issue_date_onehoter.transform(df)
#df.show(3)
#earlies_credit_mon
earlies_credit_mon_indexer = StringIndexer(inputCol='earlies_credit_mon', outputCol='earlies_credit_mon_num').fit(df)
df = earlies_credit_mon_indexer.transform(df)
earlies_credit_mon_onehoter = OneHotEncoder(inputCol='earlies_credit_mon_num', outputCol='earlies_credit_mon_vector')
df = earlies_credit_mon_onehoter.transform(df)
df.show(3)
df.withColumnRenamed("is_default", "label").show(3)

+-------+-------+----------+------------+--------+---------------+-----+---------+---------+-------------+------------------------------+---------+-----------+-----------------+-------------+--------+----------+----------+---+---------+------+---------------+--------------+-----------+------------+----------------+------------+-------------------+------------------------+----------+----------+-------------------+------------------+-----+-----------+---+---+----+----+---+---+-----+---------+-------------+-------------+----------------+-------------+----------------+-----------------+--------------------+------------+---------------+-------------+----------------+--------------+-----------------+----------------------+-------------------------+
|loan_id|user_id|total_loan|year_of_loan|interest|monthly_payment|class|sub_class|work_type|employer_type|                      industry|work_year|house_exist|house_loan_status|censor_status|marriage|offsprings|issue_date|use|post_code|region|debt

In [4]:
tmpCols=['total_loan', 'year_of_loan', 'interest', 'monthly_payment', 'class_vector','sub_class_vector','work_type_vector','employer_type_vector','industry_vector','issue_date_vector','earlies_credit_mon_vector','house_exist','house_loan_status','censor_status','marriage','offsprings','use','post_code','region','debt_loan_ratio','del_in_18month','scoring_low','scoring_high','pub_dero_bankrup','early_return','early_return_amount','early_return_amount_3mon','recircle_b','recircle_u','initial_list_status','title','policy_code','f0','f1','f2','f3','f4','f5']
for i in tmpCols:
    if "vector" in i:
        print("")
    else:
        df = df.withColumn(i, df[i].cast('double'))
#df = df.withColumn("total_loan", df["total_loan"].cast(IntegerType()))
df = df.withColumn('label', df['label'].cast(IntegerType()))
df.show(3)








+-------+-------+----------+------------+--------+---------------+-----+---------+---------+-------------+------------------------------+---------+-----------+-----------------+-------------+--------+----------+----------+---+---------+------+---------------+--------------+-----------+------------+----------------+------------+-------------------+------------------------+----------+----------+-------------------+------------------+-----+-----------+----+----+----+----+----+----+-----+---------+-------------+-------------+----------------+-------------+----------------+-----------------+--------------------+------------+---------------+-------------+----------------+--------------+-----------------+----------------------+-------------------------+
|loan_id|user_id|total_loan|year_of_loan|interest|monthly_payment|class|sub_class|work_type|employer_type|                      industry|work_year|house_exist|house_loan_status|censor_status|marriage|offsprings|issue_date|use|post_code|

In [5]:
data = df.drop('label')
feas = data.columns
df_assembler = VectorAssembler(inputCols=['total_loan', 'year_of_loan', 'interest', 'monthly_payment', 'class_vector','sub_class_vector','work_type_vector','work_year_vector','employer_type_vector','industry_vector','issue_date_vector','earlies_credit_mon_vector','house_exist','house_loan_status','censor_status','marriage','offsprings','use','post_code','region','debt_loan_ratio','del_in_18month','scoring_low','scoring_high','pub_dero_bankrup','early_return','early_return_amount','early_return_amount_3mon','recircle_b','recircle_u','initial_list_status','title','policy_code','f0','f1','f2','f3','f4','f5'],outputCol='features')
print(df_assembler)
data = df_assembler.transform(df)
data.show()

VectorAssembler_8c4951986be5
+-------+-------+----------+------------+--------+---------------+-----+---------+---------+--------------+------------------------------+---------+-----------+-----------------+-------------+--------+----------+----------+---+---------+------+---------------+--------------+-----------+------------+----------------+------------+-------------------+------------------------+----------+----------+-------------------+------------------+-------+-----------+----+----+----+----+----+----+-----+---------+-------------+-------------+----------------+-------------+----------------+-----------------+--------------------+------------+---------------+-------------+----------------+--------------+-----------------+----------------------+-------------------------+--------------------+
|loan_id|user_id|total_loan|year_of_loan|interest|monthly_payment|class|sub_class|work_type| employer_type|                      industry|work_year|house_exist|house_loan_status|censor_statu

In [6]:
data_set = data.select(['features', 'label'])
train_df, test_df = data_set.randomSplit([0.8, 0.2])
#print(' train_df shape : (%d , %d)'%(train_df.count(), len(train_df.columns)))
#print(' test_df  shape: :(%d , %d)'%(test_df.count(), len(test_df.columns)))

In [7]:
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[2, 2, 2], blockSize=1, seed=123)
mlpc_reg = mlp.fit(train_df)

In [10]:
test0=test_df.select('features')
#mlpc_reg.transform(test0).head().prediction
predict_data= mlpc_reg.transform(test0)

In [16]:
predict_data.printSchema()
mlpc_reg.transform(test0).select("features", "prediction").show()

root
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



Py4JJavaError: An error occurred while calling o908.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 55.0 failed 1 times, most recent failure: Lost task 0.0 in stage 55.0 (TID 55, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPl

In [31]:


#traing_err = predict_data.filter(predict_data['label'] != predict_data['prediction']).count() 
#total = predict_data.count()
#dt_scores = float(traing_err)/total
#print(traing_err, total, float(traing_err)/total)

In [39]:
predict_data.printSchema()
#predict_data.show()
print(type(predict_data))
print(type(test_df))
print(predict_data.select('label').collect())
print(predict_data.select('prediction').collect())
#tp=predict_data.filter("label==1 and prediction==1").count()
#tn=predict_data.filter("label==0 and prediction==0").count()
#fp=predict_data.filter("label==0 and prediction==1").count()
#fn=predict_data.filter("label==1 and prediction==0").count()
#print(tp)

root
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
[Row(label=1), Row(label=0), Row(label=1), Row(label=0), Row(label=0), Row(label=0), Row(label=1), Row(label=0), Row(label=1), Row(label=1), Row(label=0), Row(label=0), Row(label=1), Row(label=1), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=1), Row(label=1), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=1), Row(label=0), Row(label=0), Row(label=1), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=1), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=0), Row(label=1), Row(label=1), 

Py4JJavaError: An error occurred while calling o1872.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 90.0 failed 1 times, most recent failure: Lost task 0.0 in stage 90.0 (TID 77, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:260)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:252)
	

In [11]:
test0=test_df.drop('label')
mlpc_reg.transform(test0).head().prediction
predictions = mlpc_reg.transform(test_df)

Py4JJavaError: An error occurred while calling o768.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 53.0 failed 1 times, most recent failure: Lost task 0.0 in stage 53.0 (TID 52, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.Sp

In [9]:
tp=predictions.filter("label==1 and prediction==1").count()
tn=predictions.filter("label==0 and prediction==0").count()
fp=predictions.filter("label==0 and prediction==1").count()
fn=predictions.filter("label==1 and prediction==0").count()
print(tp)

Py4JJavaError: An error occurred while calling o705.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 51, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.

In [ ]:
print('test accuracy is : %f'%((tp+tn)/(tp+tn+fp+fn)))
recal=tp/(tp+fn)
prec=tp/(tp+fp)
print('test recall is : %f'%(recal))
print('test precision is : %f'%(prec))
print('test f1-score is : %f'%(2*recal*prec/(prec+recal)))


In [26]:
sc.stop()